Import stuff

In [107]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

#pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

Import Envelope Lattice/streetnetwork

In [108]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
envelope_lattice.shape

env_cens = envelope_lattice.centroids_threshold(-1)

print (env_cens)

# import the streetnetwork as a point cloud (workshop file, temporary)
street_path = os.path.relpath("../data/meshes/main_street_points.csv")
street_pc = tg.cloud_from_csv(street_path)

[[ -40. -100.    0.]
 [ -40. -100.   10.]
 [ -40. -100.   20.]
 ...
 [ 110.  -20.   20.]
 [ 110.  -20.   30.]
 [ 110.  -20.   40.]]


Calculate maximum/minimum x and y

In [122]:
border_lattice = envelope_lattice * 0.0
# border_lattice[:,:,1:-1] += 1.0
border_lattice[:,1:-1,:] += 1.0
border_lattice[1:-1,:,:] += 1.0
border_lattice = border_lattice != 2
border_lattice.centroids

cloud([[ -40., -100.,    0.],
       [ -40., -100.,   10.],
       [ -40., -100.,   20.],
       [ -40., -100.,   30.],
       [ -40., -100.,   40.],
       [ -40.,  -90.,    0.],
       [ -40.,  -90.,   10.],
       [ -40.,  -90.,   20.],
       [ -40.,  -90.,   30.],
       [ -40.,  -90.,   40.],
       [ -40.,  -80.,    0.],
       [ -40.,  -80.,   10.],
       [ -40.,  -80.,   20.],
       [ -40.,  -80.,   30.],
       [ -40.,  -80.,   40.],
       [ -40.,  -70.,    0.],
       [ -40.,  -70.,   10.],
       [ -40.,  -70.,   20.],
       [ -40.,  -70.,   30.],
       [ -40.,  -70.,   40.],
       [ -40.,  -60.,    0.],
       [ -40.,  -60.,   10.],
       [ -40.,  -60.,   20.],
       [ -40.,  -60.,   30.],
       [ -40.,  -60.,   40.],
       [ -40.,  -50.,    0.],
       [ -40.,  -50.,   10.],
       [ -40.,  -50.,   20.],
       [ -40.,  -50.,   30.],
       [ -40.,  -50.,   40.],
       [ -40.,  -40.,    0.],
       [ -40.,  -40.,   10.],
       [ -40.,  -40.,   20.],
       [ -

In [123]:
p = pv.Plotter(notebook=True)

base_lattice = border_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(247.21077512227743, 152.21077512227743, 232.21077512227743),
 (35.0, -60.0, 20.0),
 (0.0, 0.0, 1.0)]

In [68]:
#minimum values for x and y
xmin, ymin, zmin = env_cens.min(axis=0)

print (xmin)
print (ymin)

#maximum values for x and y
xmax, ymax, zmax = env_cens.max(axis=0)

print (xmax)
print (ymax)

-40.0
-100.0
110.0
-20.0


Calculate points on street

In [154]:
#calculate the x coördinates
x = np.arange (xmin, xmax)

#calculate the y coördinates
y = np.arange (ymin, ymax)

X = np.tile(x,(len(y), 1) )
Y = np.tile(y,(len(x), 1) )
#print(Y.T.shape)
#print(X.shape)
Z = np.zeros((len(y), len(x)))
#print(Z.shape)
something = np.stack([X.flatten(),Y.T.flatten(),Z.flatten()]).T
print (something)

[[ -40. -100.    0.]
 [ -39. -100.    0.]
 [ -38. -100.    0.]
 ...
 [ 107.  -21.    0.]
 [ 108.  -21.    0.]
 [ 109.  -21.    0.]]


Calculate distance

In [152]:
# extracting the centroid of all voxels
env_cens = something
#print(env_cens.size)
# initializing the distance matrix
dist_m = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for street_point in street_pc:
        # find the difference vector
        diff = voxel_cen - street_point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_m.append(dist_v)
# change the distance matrix type, from list to array
dist_m = np.array(dist_m)
#print(dist_m.size, dist_m.shape)

# find the distance to the closest street point for each voxel
min_dist = dist_m.min(axis=1)

np_lat = np.array(env_cens)

#np.append(np_lat, min_dist)

loudness = []

#for count in range(len(np_lat)):
#   new_array.append(np.append(np_lat[count], min_dist[count]))

for count, lat_val in enumerate(np_lat):
   loudness.append(np.append(lat_val, min_dist[count]))

loudness

[array([ -40.        , -100.        ,    0.        ,  150.96039636]),
 array([ -39.        , -100.        ,    0.        ,  149.96042568]),
 array([ -38.        , -100.        ,    0.        ,  148.96045539]),
 array([ -37.       , -100.       ,    0.       ,  147.9604855]),
 array([ -36.        , -100.        ,    0.        ,  146.96051603]),
 array([ -35.        , -100.        ,    0.        ,  145.96054697]),
 array([ -34.        , -100.        ,    0.        ,  144.96057833]),
 array([ -33.        , -100.        ,    0.        ,  143.96061013]),
 array([ -32.        , -100.        ,    0.        ,  142.96064238]),
 array([ -31.        , -100.        ,    0.        ,  141.96067508]),
 array([ -30.        , -100.        ,    0.        ,  140.96070825]),
 array([ -29.        , -100.        ,    0.        ,  139.96074189]),
 array([ -28.        , -100.        ,    0.        ,  138.96077601]),
 array([ -27.        , -100.        ,    0.        ,  137.96081063]),
 array([ -26.        , -